# Connecting to the Prompt Hub

We can connect our application to LangSmith's Prompt Hub, which will allow us to test and iterate on our prompts within LangSmith, and pull our improvements directly into our application.

### Setup

In [1]:
import os

In [2]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../../.env", override=True)

True

### Pull a prompt from Prompt Hub

Pull in a prompt from Prompt Hub by pasting in the code snippet from the UI.

In [5]:
from langchain import hub
prompt = hub.pull("pirate_friend")

Let's see what we pulled - note that we did not get the model, so this is just a StructuredPrompt and not runnable.

In [6]:
prompt

ChatPromptTemplate(input_variables=['language', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'pirate_friend', 'lc_hub_commit_hash': '6aafbcbfe3531a2f3f9898ad6d1a5d927233d453a0508cf899b15271548cdd33'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='You are a Henry Avery, a from the 1600s. You only speak {language}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])

Cool! Now let's hydrate our prompt by calling .invoke() with our inputs

In [7]:
hydrated_prompt = prompt.invoke({"question": "Are you a captain yet?", "language": "Spanish"})
hydrated_prompt

ChatPromptValue(messages=[SystemMessage(content='You are a Henry Avery, a from the 1600s. You only speak Spanish', additional_kwargs={}, response_metadata={}), HumanMessage(content='Are you a captain yet?', additional_kwargs={}, response_metadata={})])

And now let's pass those messages to OpenAI and see what we get back!

In [8]:
from openai import OpenAI
from langsmith.client import convert_prompt_to_openai_format

openai_client = OpenAI()

# NOTE: We can use this utility from LangSmith to convert our hydrated prompt to openai format
converted_messages = convert_prompt_to_openai_format(hydrated_prompt)["messages"]

openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=converted_messages,
    )

ChatCompletion(id='chatcmpl-CNdSNOoqn1YThcBkurtXNiMiihyhm', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Sí, soy un capitán. He navegado los mares en busca de tesoros y aventuras. ¿Te gustaría saber más sobre mis travesías?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1759749251, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=31, prompt_tokens=35, total_tokens=66, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

##### [Extra: LangChain Only] Pulling down the Model Configuration

We can also pull down the saved model configuration as a LangChain RunnableBinding when we use `include_model=True`. This allows us to run our prompt template directly with the saved model configuration.

In [11]:
from langchain import hub
prompt = hub.pull("pirate_friend", include_model=True)

/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/json/decoder.py:337: UserWarning: WARNING! extra_headers is not default parameter.
                extra_headers was transferred to model_kwargs.
                Please confirm that extra_headers is what you intended.
  obj, end = self.raw_decode(s, idx=_w(s, 0).end())


In [12]:
prompt

ChatPromptTemplate(input_variables=['language', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'pirate_friend', 'lc_hub_commit_hash': '6aafbcbfe3531a2f3f9898ad6d1a5d927233d453a0508cf899b15271548cdd33'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='You are a Henry Avery, a from the 1600s. You only speak {language}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x10c92b6d0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x10e86f250>, root_client=<openai.OpenAI object at 0x10c92b8b0>, root_async_client=<openai.AsyncOpenAI object at 0x10e85af40>, model_name='o4-mini', model_kwargs

Test out your prompt!

In [13]:
prompt.invoke({"question": "Are you a captain yet?", "language": "Spanish"})

AIMessage(content='Sí, ya ostento el grado de capitán. Soy Henry Avery, capitán de La Fancy, nave temida en los mares del Índico tras la captura del Ganj-i-Sawai.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 314, 'prompt_tokens': 34, 'total_tokens': 348, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'o4-mini-2025-04-16', 'system_fingerprint': None, 'id': 'chatcmpl-CNdUJWR5ibpHPtcN9LUYVF3NF1zBe', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--a72b4c59-a3a7-4c75-822c-ee73924cf5f2-0', usage_metadata={'input_tokens': 34, 'output_tokens': 314, 'total_tokens': 348, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 256}})

### Pull down a specific commit

Pull down a specific commit from the Prompt Hub by pasting in the code snippet from the UI.

In [14]:
from langchain import hub
prompt = hub.pull("pirate_friend:a1b6ce7b")

Run this commit!

In [15]:
from openai import OpenAI
from langsmith.client import convert_prompt_to_openai_format

openai_client = OpenAI()

hydrated_prompt = prompt.invoke({"question": "What is the world like?", "language": "English"})
# NOTE: We can use this utility from LangSmith to convert our hydrated prompt to openai format
converted_messages = convert_prompt_to_openai_format(hydrated_prompt)["messages"]

openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=converted_messages,
    )

ChatCompletion(id='chatcmpl-CNdWBsn0EO1uBGT5UO0FioVrYeMvq', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Arrr, matey! The world of 2500 be a fantastical place, full of wonders and challenges. Technology be advanced beyond yer wildest dreams, where ships sail the skies and beneath the ocean waves, powered by clean energy and artificial intelligence. The seas be teemin' with life and mystery, but also with the remnants of old empires and the ruins of ancient cities.\n\nSociety be a mix of cultures, an’ folks from all corners of the galaxy be minglin' together. There be a strong emphasis on sustainability and protectin' the environment, as humans learned from the mistakes of the past. However, tensions still rise over resources, territory, and treasure, makin' piracy a still-thrivin' trade for those brave enough to chart the unknown.\n\nIn the skies and on the waves, adventure awaits, and every horizon holds the promise of fortune—or

### Uploading Prompts

You can also easily update your prompts in the hub programmatically.



In [16]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client

client=Client()

french_prompt = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation.

Your users can only speak French, make sure you only answer your users with French.

Conversation: {conversation}
Context: {context} 
Question: {question}
Answer:"""

french_prompt_template = ChatPromptTemplate.from_template(french_prompt)
client.push_prompt("french-rag-prompt", object=french_prompt_template)

'https://smith.langchain.com/prompts/french-rag-prompt/75567b82?organizationId=b596d003-75cc-4859-ab5f-9c3358306b32'

You can also push a prompt as a RunnableSequence of a prompt and a model. This is useful for storing the model configuration you want to use with this prompt. The provider must be supported by the LangSmith playground.

In [17]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client
from langchain_openai import ChatOpenAI

client=Client()
model = ChatOpenAI(model="gpt-4o-mini")

french_prompt = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation.

Your users can only speak French, make sure you only answer your users with French.

Conversation: {conversation}
Context: {context} 
Question: {question}
Answer:"""
french_prompt_template = ChatPromptTemplate.from_template(french_prompt)
chain = french_prompt_template | model
client.push_prompt("french-runnable-sequence", object=chain)

'https://smith.langchain.com/prompts/french-runnable-sequence/f1499fb1?organizationId=b596d003-75cc-4859-ab5f-9c3358306b32'